In [1]:
import json
from helper import *

import requests
import re
import cv2
from PIL import Image
from io import BytesIO
import time
import numpy as np
import pytesseract
import os
import sys

import itertools
import pdb



In [2]:
pd.options.display.max_rows = 100
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [3]:
def findsubsets(s, n): 
    #return list(itertools.combinations(s, n)) 
    return list(map(set, itertools.combinations(s, n))) 
s = {1, 2, 3} 
n = 2
  
print(findsubsets(s, n))

[{1, 2}, {1, 3}, {2, 3}]


In [4]:
#pd.set_option('precision', 2)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)

In [5]:
def mudancas_morfologicas(imagem,x=3,tipo='erode'):


    im_gray = np.array(imagem)
    kernel = np.ones((x, x), np.uint8)
    if tipo == 'erode' or tipo == 'dilate':
        im_gray = cv2.erode(im_gray, kernel, iterations=1)
        im_gray = cv2.dilate(im_gray, kernel, iterations=1)
    if tipo == 'open' or tipo == 'close':
        im_gray = cv2.morphologyEx(im_gray, cv2.MORPH_OPEN, kernel)
        im_gray = cv2.morphologyEx(im_gray, cv2.MORPH_CLOSE, kernel)
        
    resposta = pytesseract.image_to_string(im_gray, lang='eng',config='-c tessedit_char_whitelist=0123456789').replace(' ', '').replace('.', '').replace(',', '').replace('-', '')
    if not resposta:
        if x == 5:
            return 'erro'
        elif x == 3:
            mudancas_morfologicas(imagem, x=2, tipo='erode')
        elif x == 2:
            mudancas_morfologicas(imagem, x=4, tipo='erode')
        elif x == 4:
            mudancas_morfologicas(imagem, x=1, tipo='erode')
        else:
            mudancas_morfologicas(imagem, x=5, tipo='erode')
    else:
        return resposta


def captcha():
    url = "http://cvmweb.cvm.gov.br/SWB/Sistemas/SCW/CPublica/FormBuscaPartic.aspx?CD_ERRO=4&TpConsulta=5"

    r1 = requests.get(url)
    cookies_dict = r1.cookies.get_dict()
    __VIEWSTATE = re.findall('name="__VIEWSTATE" id="__VIEWSTATE" value="(.*?)" />\r\n\r\n<input', r1.text)[0]
    __EVENTVALIDATION = re.findall('name="__EVENTVALIDATION" id="__EVENTVALIDATION" value="(.*?)"', r1.text)[0]
    __VIEWSTATEGENERATOR = re.findall('id="__VIEWSTATEGENERATOR" value="(.*?)"', r1.text)[0]
    v1 = re.findall(r"<img src=\'RandomTxt.aspx(.*?)\'><br>", r1.text)[0]
    r2 = requests.get("http://cvmweb.cvm.gov.br/SWB/Sistemas/SCW/CPublica/RandomTxt.aspx" + v1, cookies=cookies_dict)
    imagem = Image.open(BytesIO(r2.content))
    return imagem, cookies_dict,__VIEWSTATE,__EVENTVALIDATION,__VIEWSTATEGENERATOR,mudancas_morfologicas(imagem)



In [6]:
def salva_carteira(fundo, cnpj):
    url = "http://cvmweb.cvm.gov.br/SWB/Sistemas/SCW/CPublica/FormBuscaPartic.aspx?CD_ERRO=4&TpConsulta=5"
    imagem, cookies_dict,__VIEWSTATE,__EVENTVALIDATION,__VIEWSTATEGENERATOR,resposta = captcha()
    if not resposta:
        salva_carteira(fundo, cnpj)
    else:
        print(resposta)
        url =  "https://cvmweb.cvm.gov.br/SWB/Sistemas/SCW/CPublica/ResultBuscaPartic.aspx?TpConsulta=5&CNPJNome="+str(cnpj)+"&COMPTC=&numRandom="+resposta
        params = {'__EVENTTARGET': '',
              '__EVENTARGUMENT': '',
              '__VIEWSTATE': __VIEWSTATE,
              '__VIEWSTATEGENERATOR' : __VIEWSTATEGENERATOR,
              '__EVENTVALIDATION': __EVENTVALIDATION,
              'txtCNPJNome': str(cnpj),
              'ddComptc': '',
              'numRandom': resposta,
              'btnContinuar': 'Continuar >'
              }
        r3 = requests.post(url, cookies=cookies_dict, data=params)



        CNPJ = re.findall(r'<span id="lbNrPfPj">(.* ?)</span>', r3.text)
        if not CNPJ:
            salva_carteira(fundo, cnpj)
        else:

            url = re.findall(r'PK_PARTIC=(.*?)&amp;COMPTC=', r3.text)[0]
            url = 'https://cvmweb.cvm.gov.br/SWB/Sistemas/SCW/CPublica/CDA/CPublicaCDA.aspx?PK_PARTIC=' + url + '&COMPTC='
            if CNPJ[0].replace('.','').replace('-','').replace('/','') != cnpj:
                print(cnpj,' Deu erro no CNPJ que voltou ',CNPJ[0].replace('.','').replace('-','').replace('/',''))
                #print('Deu erro no CNPJ que voltou')
            else:
                opcoes = re.findall(r'<option(.*?)value="(.*?)">(.*?)</option>', r3.text)
                if opcoes[-1][-1] == 'Anteriores':
                    opcoes = opcoes[:-1]
                # Testa se o fundo deixou de mandar algum demonstrativo mensal
                if len(opcoes) != (int(opcoes[0][-1].split('/')[0]) - int(opcoes[-1][-1].split('/')[0]) + 1) + 12 * (
                        int(opcoes[0][-1].split('/')[1]) - int(opcoes[-1][-1].split('/')[1])):
                    print('Deixaram de mandar algum demonstrativo mensal')
                out = 'htmls\\' + fundo
                if not os.path.exists(out):
                    os.makedirs(out)
                if not (os.path.isfile(out + '/' + opcoes[0][-1].split('/')[1] + opcoes[0][-1].split('/')[0] + cnpj + '.html')):
                    with open(out + '/' + opcoes[0][-1].split('/')[1] + opcoes[0][-1].split('/')[0] + cnpj + '.html', 'w') as f:
                        f.write(r3.text)


                #return
                ####
                #for n in range(1,13):
                #for n in range(1,len(opcoes)):
                for n in [3]:

                    __VIEWSTATE = re.findall('name="__VIEWSTATE" id="__VIEWSTATE" value="(.*?)" />\r\n\r\n<input', r3.text)[0]
                    __EVENTVALIDATION = re.findall('name="__EVENTVALIDATION" id="__EVENTVALIDATION" value="(.*?)"', r3.text)[0]
                    __VIEWSTATEGENERATOR = re.findall('id="__VIEWSTATEGENERATOR" value="(.*?)"', r3.text)[0]

                    time.sleep(3)
                    params = {
                                '__EVENTTARGET': 'ddCOMPTC',
                                '__EVENTARGUMENT': '',
                                '__LASTFOCUS': '',
                                '__VIEWSTATE': __VIEWSTATE,
                                '__VIEWSTATEGENERATOR' : __VIEWSTATEGENERATOR,
                                '__EVENTVALIDATION': __EVENTVALIDATION,
                                'ddCOMPTC': opcoes[n][1]
                    }
                    r3 = requests.post(url, cookies=cookies_dict, data=params)
                    if not (
                    os.path.isfile(out + '/' + opcoes[n][-1].split('/')[1] + opcoes[n][-1].split('/')[0] + cnpj + '.html')):
                        with open(out + '/' + opcoes[n][-1].split('/')[1] + opcoes[n][-1].split('/')[0] + cnpj + '.html',
                                  'w') as f:
                            f.write(r3.text)


In [7]:
for fundo in ['dynamo']:
    fundo = fundo.upper()
    print(fundo)
    salva_carteira(fundo, cnpj(fundo)[1])

DYNAMO
17243


In [46]:
anomes = 202003
apelido_fundo = 'DYNAMO'
carteira_pandas,fundo,fundo_cnpj,adm,adm_cnpj,pl,hora_recebimento = parse_html_tudo_int(apelido_fundo,anomes,candidatos=True)

C:\Users\luiz.cavalcante\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [47]:
list(zip(carteira_pandas.loc[3].values[16],carteira_pandas.loc[3].values[17]))

[('AFLT3', 26879475),
 ('AZEV3', 49919025),
 ('BEEF11', 45146405),
 ('BEES3', 34734908),
 ('BEES4', 35083652),
 ('BIOM11', 26879475),
 ('BPAC11', 5256215),
 ('BRGE3', 34597344),
 ('BRKM3', 9843188),
 ('BRML3', 17506672),
 ('BSEV3', 60247099),
 ('CAMB3', 39798767),
 ('CTKA4', 26879475),
 ('DTEX3', 18686266),
 ('EKTR4', 9983805),
 ('ELEK4', 8779728),
 ('ENAT3', 19094709),
 ('ENGI4', 26633626),
 ('ETER3', 68248667),
 ('FESA4', 13470824),
 ('GRND3', 23202734),
 ('IDVL4', 69886635),
 ('ISBC34', 4835776),
 ('ISUS11', 6399875),
 ('ITSA3', 17559456),
 ('JBDU3', 84813877),
 ('JFEN3', 64709847),
 ('JHSF3', 49919025),
 ('K1EY34', 3903409),
 ('KEPL3', 7189983),
 ('KNRE11', 34191113),
 ('LPSB3', 49215940),
 ('MRVE3', 14274231),
 ('PSVM11', 71899830),
 ('PTNT4', 43898640),
 ('RANI3', 57662240),
 ('RAPT3', 35083652),
 ('RIOT34', 828787),
 ('ROMI3', 21921780),
 ('RPAD5', 21569949),
 ('RSID3', 40350251),
 ('SANB3', 13316813),
 ('SGPS3', 35876096),
 ('TASA3', 48803516),
 ('TESA12', 79057279),
 ('TIET11'

In [48]:
carteira_pandas

,aserestudado,ativo1,ativo2,ativo3,ativo4,classificacao,emp_ligada,vendas_quant,vendas_montante,aquis_quant,aquis_montante,pf_quant,pf_valor_custo,pf_valor_mercado,porcent_partrimonio_liquido,fechado,candidatos,cand_pf_quant
59,57726459,Títulos Públicos,Cod. SELIC:,210100,Venc.:01/09/2021,Para negociacao,NaN,0.0,0.000000e+00,2722.0,2.872584e+09,4710.0,NaN,4980445594,1.383,False,"[A1LK34, AFLT3, ALSO3, AZEV4, BBAS3, BGIP3, BI...","[320492, 7662224, 1872348, 22846081, 1785746, ..."
57,57726460,Operações Compromissadas,Cod. SELIC:,950199,Venc.:01/01/2023,Para negociacao,Nao,NaN,NaN,NaN,NaN,431224.0,NaN,48538705352,13.479,False,"[A1PA34, AZEV4, AZUL4, BEES4, BKBR3, BPAC3, BR...","[18769801, 222654612, 27657382, 97467280, 5357..."
58,57726461,Operações Compromissadas,Cod. SELIC:,100000,Venc.:01/07/2020,Para negociacao,Nao,NaN,NaN,NaN,NaN,54443.0,NaN,5393189484,1.498,False,"[A1TT34, ABCB4, AMAR3, ATMP3, AZEV3, BPAN4, CG...","[255311, 4073406, 11724325, 9987388, 15409113,..."
32,57726462,Cotas de Fundos,NaN,ISHARES BM&FBOVESPA SMALL CAP FUNDO DE ÍNDICE,NaN,NaN,Nao,NaN,NaN,NaN,NaN,80003.0,NaN,629703613,0.175,False,"[ABCB4, AMAR3, AZEV4, BEES3, BRGE12, BRGE3, BR...","[475607, 1368921, 2888549, 1251896, 1298358, 1..."
29,57726463,Ações e outros TVM cedidos em empréstimo,Cod. Ativo:,MRVE3,Dt. Ini. Vigen.:23/07/2007,Para negociacao,Nao,NaN,NaN,NaN,NaN,159730.0,NaN,195509520,0.054,False,"[A1LB34, AFLT3, ALUP4, AZEV4, BDLL4, BEEF3, BE...","[6836, 300784, 326939, 896833, 278901, 245924,..."
11,57726464,Ações,Cod. Ativo:,ALPA4,Dt. Ini. Vigen.:19/03/1973,Para negociacao,Nao,3436.0,7.546452e+06,477033.0,1.040347e+09,4636742.0,NaN,10738694472,2.982,False,"[AALL34, ALPA4, AMAR3, BPAN4, BRML3, BSEV3, CA...","[1691133, 4636742, 23344988, 22703371, 1076021..."
15,57726465,Ações,Cod. Ativo:,ANIM3,Dt. Ini. Vigen.:28/10/2013,Para negociacao,Nao,0.0,0.000000e+00,367699.0,6.626007e+08,2490151.0,NaN,4748717957,1.319,False,"[ABCB4, AFLT3, AHEB5, ALUP4, AMAR3, ANIM3, ATT...","[3586645, 7305720, 2371987, 7941000, 10323300,..."
10,57726466,Ações,Cod. Ativo:,B3SA3,Dt. Ini. Vigen.:26/03/2018,Para negociacao,Nao,1231428.0,4.925127e+09,412025.0,1.469145e+09,3032596.0,NaN,10887019640,3.023,False,"[AFLT3, ALZR11, AMAR3, APER3, AZUL4, B3SA3, BI...","[16749261, 1079526, 23667434, 7262855, 6203430..."
2,57726467,Ações,Cod. Ativo:,GGBR4,Dt. Ini. Vigen.:01/02/2001,Para negociacao,Nao,0.0,0.000000e+00,4337653.0,4.372902e+09,18485425.0,NaN,18577852125,5.159,False,"[AALR3, AFLT3, ATMP3, BEEF11, BEES4, BIOM11, B...","[20642058, 28581311, 34403430, 48004786, 37304..."
23,57726468,Ações,Cod. Ativo:,GOAU3,Dt. Ini. Vigen.:27/08/1973,Para negociacao,Nao,0.0,0.000000e+00,0.0,0.000000e+00,2088571.0,NaN,868845536,0.241,False,"[AALR3, ALUP4, AZEV3, AZEV4, BAZA3, BEEF11, BM...","[965384, 1452919, 2482416, 3985530, 297448, 22..."
